# Experiment Workbench

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds
#from tensorflow import keras
keras = tf.keras
from keras.layers import ReLU
from keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 # Import the necessary MLTK APIs
from mltk.core import view_model,  profile_model # summarize_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.utils.utils import parse_model_name
from workbench.data.data import get_vvw_dataset


#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import deeplake


d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.2.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Confirm that TensorFlow can access GPU
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.10.0
Found GPU at: /device:GPU:0


In [3]:
global model_name
#model_name = "efficientNetB0_1_96_c3_o3_keras"
#model_name = "shufflenetv2tinys_0.2_96_c3_o3_f4l1024"2
model_name = "mobilenetv2_0.35_96_c3_o2_l5"
#model_name = "mobilenetv2_0.35_96_c3_o2_keras"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"
#model_name = "mobilenetvme_0.35_96_c3_o3_l5"
#model_name = "shufflenetv2tiny_0.2_96_c3_o3_f4l1024"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"#, "MobilenetV3large_1_224_c3_o3_keras"# ,


In [4]:
# tf.compat.v1.disable_eager_execution()

# DANGER ZONE: Disable warning messages

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [5]:
tf.keras.backend.clear_session()

https://www.tensorflow.org/api_docs/python/tf/random/set_seed  

Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed.

Its interactions with operation-level seeds is as follows:

1. If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.  
2. If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.  
3. If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.  
4. If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [6]:
seed_1 = 1
seed_2 = 15
seed_3 = 30
seed_4 = 42
seed_5 = 75

seed = seed_2

# set the random seeds
#os.environ["TF_CUDNN_DETERMINISTIC"]= "1"
# random.seed(seed)
# np.random.seed(seed)
# tf.random.set_seed(seed) # setting tensorflow global seed
tf.keras.utils.set_random_seed(seed)
# tf.config.experimental.enable_op_determinism() 

# Node: 'gradient_tape/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/UnsortedSegmentSum'
#Deterministic GPU implementation of unsorted segment reduction op not available.
#	 [[{{node gradient_tape/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/UnsortedSegmentSum}}]] [Op:__inference_train_function_10280]

In [7]:
models_dir = initialize()

# Getting datasets from deeplake

In [8]:
# ds = deeplake.load('hub://activeloop/plantvillage-without-augmentation')#   61486 images, 39 different plant diseases
# len_plants  = 61486
# plant_classes = 39
#ds = deeplake.load("hub://activeloop/places205") # this dataset is huge! 2.5Mio Images

In [9]:
# ds.visualize()

In [10]:
# ds_tensorflow = ds.tensorflow()

In [11]:
# #deeplake.__version__
# image_count = len_plants
# train_size = int(0.8 * image_count)
# val_size = int(0.1 * image_count)
# test_size = int(0.1 * image_count)

# ds_tensorflow = ds_tensorflow.shuffle(image_count)
# test_ds = ds_tensorflow.take(test_size)
# train_ds = ds_tensorflow.skip(test_size)
# val_ds = ds_tensorflow.take(val_size)
# train_ds = ds_tensorflow.skip(val_size)

In [12]:
# train_ds.cardinality()

In [13]:
# for sample in train_ds.as_numpy_iterator():
#     print(sample)
#     break 

In [14]:
# train_iter = train_ds.as_numpy_iterator()

In [15]:
# element = next(train_iter)

In [16]:
# for element in train_ds:
#     print(element)
#     break
# element["images"].numpy() # image numpy array
# element["labels"].numpy() # labels numpy array

In [17]:
# i = 0
# for elem in ds_tensorflow: 
#   i = i + 1

In [18]:
# train_ds.take(1)

In [19]:
#print(example)

In [20]:
# train_ds.options

In [21]:
# train_ds.__dict__

In [22]:
# type(train_ds)

In [23]:
# help(train_ds)

In [24]:
# train_ds_iter = iter(train_ds)
# next(train_ds_iter)

In [25]:
#next(train_ds)

# Visual Wake Words dataset


In [26]:
vvw_path = Path.cwd().parent.joinpath("person_detection","datavisualwakewords")
vvw_path.exists()

True

# Get the model

In [27]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [28]:
model = keras.models.load_model(models_tf_path)

In [29]:
global base_model_name
global alpha
global resolution
global channels
global classes
global variation
global early_stopping_patience

In [30]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")

In [31]:
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))

In [32]:
# Start a Tensorboard session
%load_ext tensorboard

# Set training parameters

In [33]:
#os.environ['WANDB_NOTEBOOK_NAME'] = 'Experiment Workbench'

IMG_HEIGHT = resolution
IMG_WIDTH = resolution
BATCH_SIZE = 96
EPOCHS = 30
#LOGGING_STEPS = 64
MOMENTUM = 0.9
LR = 0.045
DROPOUT = 0.2
early_stopping_patience = 30

PROJECT = base_model_name
ENTITY = "susbrock"
#PROJECT = "tiny_cnn troubleshooting"

shuffle_seed = seed

# Prepare the Lemon Quality Dataset

In [34]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [35]:
dataset_path

WindowsPath('i:/tinyml/tiny_cnn/datasets/lemon_dataset')

In [36]:
def get_lemon_quality_dataset(dataset_path, img_width, img_height, batch_size, channels, normalize=True):
    """ Fetches the lemon quality dataset and prints dataset info. It normalizes the image data to range [0,1] by default.

    Args: 
        dataset_path (Path): the file location of the dataset. Subfolders "train", "test", and "val" are expected.
        normalize (boolean): Normalizes the image data to range [0, 1]. Default: True

    Returns:
        (train_ds, val_ds, test_ds, class_names) (tuple(tf.datasets)): Tensorflow datasets for train, validation and test.
    
    """
    if dataset_path.exists():
        try:
            train_dir = dataset_path.joinpath("train")
            val_dir = dataset_path.joinpath( "val")
            test_dir = dataset_path.joinpath( "test")
        except:
            print(f"Please check the folder structure of {dataset_path}.")
            raise

    channels = int(channels.strip("c"))
    if channels==1:
        color_mode = "grayscale"
    else:
        color_mode = "rgb" 
    print(f"Color mode: {color_mode}")

    # create the labels list to avoid inclusion of .ipynb checkpoints
    #labels = ["bad_quality", "empty_background", "good_quality"]

    print("Preparing training dataset...")        
    train_ds = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=((img_height, img_width)),
        #labels=labels,
        batch_size=batch_size,
        color_mode=color_mode,
        shuffle=True
        )
    

    class_names = train_ds.class_names


    print("Preparing validation dataset...")    
    val_ds = tf.keras.utils.image_dataset_from_directory(
        val_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=(img_height, img_width),
        batch_size=batch_size,
        color_mode=color_mode,
        shuffle=True
        )
    

    print("Preparing test dataset...")    
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        subset=None,
        seed=shuffle_seed,
        image_size=(img_height, img_width),
        batch_size=1,
        color_mode=color_mode,
        shuffle=False
        )
    
    # Create a data augmentation stage with horizontal flipping, rotations, zooms
    data_augmentation = keras.Sequential(
        [
            tf.keras.layers.RandomFlip("horizontal"),
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomZoom(0.1),
        ]
        )

    #train_ds= train_ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE )

    
    # Normalize the data to the range [0, 1]
    if normalize:
        normalization_layer = tf.keras.layers.Rescaling(1./255, offset=-1)

        train_ds= train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        val_ds= val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
        test_ds= test_ds.map(lambda x, y: (normalization_layer(x), y)) #, num_parallel_calls=tf.data.AUTOTUNE)
    else:
        pass

    print (f"Class names: {class_names}")
    print(f"Train: {train_ds.element_spec}")
    print(f"Normalize: {normalize}")
    return (train_ds, val_ds, test_ds, class_names)

# Visual Wake Words minval


In [45]:
BASE_DIR = Path.cwd().parent.joinpath("tiny_mlperf", "vw_coco2014_96")
BASE_DIR_TEST = Path.cwd().parent.joinpath("tiny_mlperf", "vw_coco2014_96_test")
Path.exists(BASE_DIR)
validation_split = 0.1
color_mode = "rgb"

In [47]:
print("Preparing training dataset...")        
train_ds = tf.keras.utils.image_dataset_from_directory(
    BASE_DIR,
    validation_split=validation_split,
    subset="training",
    seed=shuffle_seed,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    #labels=labels,
    batch_size=BATCH_SIZE,
    color_mode=color_mode,
    shuffle=True
    )

class_names = train_ds.class_names

print("Preparing validation dataset...")        
val_ds = tf.keras.utils.image_dataset_from_directory(
    BASE_DIR,
    validation_split=validation_split,
    subset="validation",
    seed=shuffle_seed,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    #labels=labels,
    batch_size=BATCH_SIZE,
    color_mode=color_mode,
    shuffle=True
    )

print("Preparing test dataset...")        
test_ds = tf.keras.utils.image_dataset_from_directory(
    BASE_DIR_TEST,
    validation_split=None,
    #subset="validation",
    seed=shuffle_seed,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    #labels=labels,
    batch_size=BATCH_SIZE,
    color_mode=color_mode,
    shuffle=False
    )


    #   Preprocessing form Visual Wake Word Challenge:
    #   rotation_range=10,
    #   width_shift_range=0.05,
    #   height_shift_range=0.05,
    #   zoom_range=.1,
    #   horizontal_flip=True,
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomTranslation(
            height_factor = 0.05,
            width_factor = 0.05
        ),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomFlip("horizontal")
    ]
    )

train_ds= train_ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE )


# Normalize the data to the range [0, 1]

normalization_layer = tf.keras.layers.Rescaling(1./255, offset=-1)

train_ds= train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
val_ds= val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)
test_ds= test_ds.map(lambda x, y: (normalization_layer(x), y)) #, num_parallel_calls=tf.data.AUTOTUNE)
labels = class_names


Preparing training dataset...
Found 99619 files belonging to 2 classes.
Using 89658 files for training.
Preparing validation dataset...
Found 99619 files belonging to 2 classes.
Using 9961 files for validation.
Preparing test dataset...
Found 10000 files belonging to 2 classes.


In [38]:


# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#       rotation_range=10,
#       width_shift_range=0.05,
#       height_shift_range=0.05,
#       zoom_range=.1,
#       horizontal_flip=True,
#       validation_split=validation_split,
#       rescale=1. / 255)
# train_generator = datagen.flow_from_directory(
#     BASE_DIR,
#     target_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     subset='training',
#     color_mode='rgb',
#     class_mode="sparse")
# val_generator = datagen.flow_from_directory(
#     BASE_DIR,
#     target_size=(IMG_HEIGHT, IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     subset='validation',
#     color_mode='rgb',
#     class_mode="sparse")


# print(train_generator.class_indices)

Found 98658 images belonging to 2 classes.
Found 10961 images belonging to 2 classes.
{'non_person': 0, 'person': 1}


In [41]:
def train_epochs(model, train_generator, val_generator, epoch_count,
                 learning_rate):
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate),
      #loss='categorical_crossentropy',
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  history_fine = model.fit(
      train_generator,
      steps_per_epoch=len(train_generator),
      epochs=epoch_count,
      validation_data=val_generator,
      validation_steps=len(val_generator),
      batch_size=BATCH_SIZE)
  return model

In [42]:
model = train_epochs(model, train_generator, val_generator, 20, 0.001)
model = train_epochs(model, train_generator, val_generator, 10, 0.0005)
model = train_epochs(model, train_generator, val_generator, 20, 0.00025)

Epoch 1/20
 279/1028 [=======>......................] - ETA: 1:14:06 - loss: 0.7016 - accuracy: 0.5549

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  FileNotFoundError: [Errno 2] No such file or directory: 'i:\\tinyml\\tiny_mlperf\\vw_coco2014_96\\non_person\\COCO_val2014_000000565543.jpg'
Traceback (most recent call last):

  File "d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "d:\Miniconda\envs\tiny_cnn_5\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn


In [ ]:
#channels

In [ ]:
#train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, channels)

In [ ]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, channels)

In [ ]:
train_ds, val_ds, test_ds, labels = get_vvw_dataset(input_shape, BATCH_SIZE)

In [ ]:
#test_ds.element_spec

In [ ]:
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     img = tf.keras.preprocessing.image.array_to_img(
#                 images[0], scale=True
#             )
#     display(img)

In [ ]:
# class_names = labels
# print(class_names)

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")'

In [ ]:
# from PIL import Image
# img = np.random.random(size=(100, 100, 3))
# pil_img = tf.keras.utils.array_to_img(img)

In [ ]:
train_ds

In [ ]:
# 'test_ds

In [ ]:
# for image_batch, labels_batch in train_ds:
#     print(image_batch.shape)
#     print(labels_batch.shape)
#     print(labels_batch)
#     break

In [ ]:
# classes = len(labels)
# print(f"The dataset contains {classes } classes.")

# Define the model

In [48]:
logdir = os.path.join("logs", model_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
root_logdir = os.getcwd()

In [49]:
#wandb.__version__

In [50]:
tf.keras.backend.clear_session()

# optimize the data flow
AUTOTUNE = tf.data.AUTOTUNE
#train_ds = train_ds.cache().prefetch(AUTOTUNE)
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

In [51]:
#api = wandb.Api()
api = wandb.Api(timeout=19)

In [52]:
# code taken from https://www.tensorflow.org/guide/keras/custom_callback#examples_of_keras_callback_applications

class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


In [53]:
class EarlyStoppingAtMaxValAccuracy(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after max has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=30):
        super(EarlyStoppingAtMaxValAccuracy, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = 0
        self.best_epoch = 0
        self.best_epoch_loss = np.Infinity

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("val_accuracy")
        if np.greater(current, self.best):
            self.best = current
            self.best_epoch = epoch
            self.best_epoch_loss = logs.get("val_loss")
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

        metrics = dict()
        metrics["best_epoch"] = self.best_epoch
        metrics["best_val_accuracy"] = self.best
        metrics["best_epoch_loss"] = self.best_epoch_loss

        wandb.log(metrics)



    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))
        


In [54]:
os.environ["WANDB_MODE"] = "online"
#os.environ["WANDB_MODE"] = "offline"
def train_model_wandb(model):

        # solve issue from: https://github.com/wandb/wandb/issues/3536
        if len(wandb.patched["tensorboard"]) > 0:
                wandb.tensorboard.unpatch()
                
        # Configure Tensorboard root log directory to read the debugging information
        wandb.tensorboard.patch(root_logdir=root_logdir)
        # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
        # Generate run ids
        id = wandb.util.generate_id()

        run = wandb.init(
                # Set the project where this run will be logged
                project=PROJECT, 
                id = id, 
                resume=True,
                sync_tensorboard=True
                )

        # Specify the configuration variables
        config = wandb.config
        
        config.batch_size = BATCH_SIZE
        #config.dropout =DROPOUT
        config.learn_rate = LR
        config.momentum = MOMENTUM
        #config.decay = 1e-6
        config.epochs = EPOCHS
        config.classes = classes
        config.id = id
        config.seed = seed
        config.architecture = model_name
        

        # enable Tensorflow Debugging
        #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
        #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

       # optimizer = tf.keras.optimizers.SGD(learning_rate=LR, momentum=MOMENTUM)
        config.optimizer = optimizer._name

        model.compile(optimizer=optimizer,
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=0, update_freq="epoch") #, profile_batch="10, 20")
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir, histogram_freq=1)
        #wandb_callback = WandbCallback()# input_type="image", labels=labels) #, validation_data = val_ds.as_numpy_iterator())

        def lr_schedule(epoch):
                """
                Returns a custom learning rate that decreases as epochs progress.
                """
                learning_rate = LR
                if epoch > 20:
                        learning_rate = 0.0001
                tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
                return learning_rate

        def lr_vvw(epoch, lr):
                """
                Returns the learing rate schedule used in training for Visual Wake Word dataset.
                """
                lr = LR
                if epoch == 1:
                        lr = 0.045
                else:
                        lr = lr*0.98
                return lr


        lr_callback = LearningRateScheduler(lr_vvw)
        #lr_callback = LearningRateScheduler(lr_schedule)

        #best_model_path = Path(wandb.run.dir).joinpath(f"best_model")

        checkpoint = WandbModelCheckpoint("best_model",
                monitor="val_accuracy",
                save_best_only=True,
                save_freq="epoch")

        global early_stopping_patience
        early_stopping = EarlyStopping(monitor="val_accuracy", patience=early_stopping_patience)

        callbacks =[
                tensorboard_callback,
                lr_callback,
                #wandb_callback,
                WandbMetricsLogger(),
                checkpoint,
                #early_stopping,
                EarlyStoppingAtMaxValAccuracy()
        ]

        history = model.fit(train_ds,
                epochs=EPOCHS, 
                validation_data=val_ds, 
                callbacks=callbacks
        )

        #wandb.save("last_model.h5")




        #best_model = keras.models.load_model(best_model_path) # not needed due to "restore_best_weights=True"

        y_val_true = np.concatenate([y for x, y in val_ds], axis=0)
        y_val_pred = model.predict(val_ds).argmax(axis=1)

        y_test_true = np.concatenate([y for x, y in test_ds], axis=0)
        y_test_pred = model.predict(test_ds).argmax(axis=1)

        results = model.evaluate(test_ds, batch_size=BATCH_SIZE)
        print("test loss, test acc:", results)
        wandb.log({
                "test_loss" : results[0],
                "test_accuracy" : results[1]
        })

        # log data for the confusion matrix
        # wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
        #                 y_true=y_test_true[0], preds=y_test_pred,
        #                 class_names=labels)})


        run.finish()
        return history, model, run.id

In [55]:
wandb.finish()
history, model, run_id = train_model_wandb(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin


wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


Epoch 1/30
934/934 [==============================] - ETA: 0s - loss: 0.6719 - accuracy: 0.6329INFO:tensorflow:Assets written to: best_model\assets


INFO:tensorflow:Assets written to: best_model\assets
wandb: Adding directory to artifact (.\best_model)... Done. 0.0s


934/934 [==============================] - 578s 608ms/step - loss: 0.6719 - accuracy: 0.6329 - val_loss: 0.7006 - val_accuracy: 0.5964 - lr: 0.0441
Epoch 2/30
934/934 [==============================] - ETA: 0s - loss: 0.5950 - accuracy: 0.6882INFO:tensorflow:Assets written to: best_model\assets


...

wandb: Adding directory to artifact (.\best_model)... Done. 0.0s


934/934 [==============================] - 69s 74ms/step - loss: 0.3123 - accuracy: 0.8623 - val_loss: 0.5215 - val_accuracy: 0.7695 - lr: 0.0441
Epoch 30/30
934/934 [==============================] - ETA: 0s - loss: 0.3082 - accuracy: 0.8646

wandb: Adding directory to artifact (.\best_model)... Done. 0.0s


105/105 [==============================] - 4s 39ms/step - loss: 0.5320 - accuracy: 0.7777
test loss, test acc: [0.5319555401802063, 0.7777000069618225]


best_epoch,▁▂▂▃▃▅▆▆▆█████████████████████
best_epoch_loss,█▆▆▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,▁▄▄▆▆▇▇███████████████████████
epoch/accuracy,▁▃▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch/lr,▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▄▃▆▆▇▇███▇████████▇█▇▇███▇█▇▇
epoch/val_loss,█▆▇▃▄▂▂▂▂▁▂▁▁▁▂▁▂▂▁▃▁▂▂▂▂▂▃▂▃▃
test_accuracy,▁


In [ ]:
wandb.__version__

In [ ]:
#!wandb sync i:\tinyml\tiny_cnn\wandb\offline-run-20221227_091238-1vzrst0a

In [ ]:
#run_id

In [ ]:
#best_model = wandb.restore('/best_model/saved_model', run_path=f"{ENTITY}/{PROJECT}/{run_id}")

# Conversion to TFLite

In [ ]:
models_tflite_trained_path = models_dir.joinpath(model_name, f"{model_name}_trained.tflite")

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


In [ ]:

# Save the model.
with open(models_tflite_trained_path, "wb") as f:
    f.write(tflite_model)

# Conversion to TFLite with INT8 quantization

In [ ]:
# sample_iter = test_ds.as_numpy_iterator()

# for i in range(1):
#     sample = next(sample_iter)[0]
# print("Number of samples: {}".format(sample.shape[0]))

In [ ]:
# def representative_data_gen():
#     for i in range(100):
#       yield([test_ds[i].reshape(1, 1)])

In [ ]:
# representative_data_gen()

In [ ]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

In [ ]:
def convert_tflite_quant_INT8(model, data_generator):
    converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

    # set the optimization flag
    converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
    # enforce integer only quantization
    converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter_opt.inference_input_type = tf.uint8
    converter_opt.inference_output_type = tf.uint8

    # provide a representative dataset for quantization
    converter_opt.representative_dataset = data_generator

    tflite_model_opt = converter_opt.convert()

    return tflite_model_opt

In [ ]:
#tf_opt = convert_tflite_quant_INT8(model, data_generator)

# Run the TensorFlow Lite models


In [ ]:
# test_image = test_ds.take(1)
# test_image

In [ ]:
# len(test_ds)

In [ ]:
#test_gen = test_ds.unbatch().batch(1)
test_gen = test_ds.as_numpy_iterator()
#test_gen = test_gen.next() 
#test_image = test_gen.take(1)
test_image, test_label = next(test_gen)
test_image


In [ ]:
test_label

In [ ]:
#num_test_images = len(list(test_gen))

In [ ]:
#test_image.shape

In [ ]:
def tflite_predict(model_path, test_image):
    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path=str(model_path))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:  # was np.uint8
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point
        
    test_image = test_image.astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    #interpreter.set_tensor(input_details["index"], np.expand_dims(test_image[0], axis=0)) # only needed when input shape (96, 96, 3)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    prediction = output.argmax()
    print(f"Prediction: Class {prediction} derived from {output}")

    return prediction

In [ ]:
tflite_result = tflite_predict(models_tflite_opt_path, test_image)

In [ ]:

def tflite_predict_on_dataset(model_path, dataset):
    # find length of dataset
    test_gen = dataset.as_numpy_iterator()
    num_images = len(list(test_gen))

    predictions = []
    y_trues = []

    test_gen = dataset.as_numpy_iterator()
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    # iterate over the complete test_set
    for i in range(num_images):
        test_image, y_true = next(test_gen)
        prediction = tflite_predict(model_path, test_image)
        predictions.append(prediction)
        y_trues.append(y_true[0])
        #accuracy.update_state(y_true, prediction) # TODO: correct accuracy
        print(f"{i}, {test_image.shape} - true label: {y_true[0]} vs {tflite_result}")

    #accuracy = (np.sum(predictions == y_trues) * 100) / num_images
    print(f"Accuracy: {accuracy.result()} - (Number of test samples: {num_images})")
    return predictions, y_trues    

In [ ]:
#preds, trues = tflite_predict_on_dataset(models_tflite_opt_path, test_ds)

In [ ]:
# accuracy = (np.sum(preds == trues) * 100) / num_test_images
# accuracy

# Model evaluation

In [ ]:
# Evaluate the model on the test data using `evaluate`
# print("Evaluate on test data")
# results = model.evaluate(test_ds, batch_size=BATCH_SIZE)
# print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
# print("Generate predictions for 3 samples")
# predictions = model.predict(x_test[:3])
# print("predictions shape:", predictions.shape)

In [ ]:
# entity = "susbrock"


# run = api.run(f"{entity}/{PROJECT}/{run_id}")
# run.summary["test_accuracy"] = results[1]
# run.summary["test_loss"] = results[0]
# run.summary.update()

In [ ]:
# test_predictions = model.predict(test_ds)
# test_predictions

In [ ]:
# top_pred_ids = test_predictions.argmax(axis=1)
# len(top_pred_ids)

In [ ]:
# top_pred_ids

In [ ]:
# y_true = [y for x, y in test_ds]
# y_true

In [ ]:
# y_true = np.concatenate([y for x, y in test_ds], axis=0)
# len(y_true)

In [ ]:
# confusion_mtx = tf.math.confusion_matrix(y_true, top_pred_ids, num_classes=classes)
#     # list(ds_test.map(lambda x, y: y)),
#     # predict_class_label_number(test_data),
#     # num_classes=len(label_names))
    
# confusion_mtx

In [ ]:
def show_confusion_matrix(cm, labels):
  plt.figure(figsize=(6, 6))
  sns.heatmap(cm, xticklabels=labels, yticklabels=labels, 
              annot=True, fmt='g')
  plt.xlabel('Prediction')
  plt.ylabel('Label')
  plt.show()
  return plt

In [ ]:
# my_plot = show_confusion_matrix(confusion_mtx, labels)

In [ ]:
#my_plot.show()

## Code reserved for troubleshooting

In [ ]:
# def mobilenet_v1_keras(input_shape, classes=classes, alpha=alpha):
#     model = tf.keras.applications.mobilenet.MobileNet(
#         input_shape=input_shape,
#         alpha=alpha,
#         depth_multiplier=1,
#         dropout=0.001,
#         include_top=True,
#         weights=None, #'imagenet'
#         input_tensor=None,
#         pooling=None,
#         classes=classes,
#         classifier_activation='softmax',
#         #**kwargs
#     )

#     #model._name = model.name + "_keras" # model.name cannot be overritten

#     return model
#     #model = mobilenet_v1_keras((IMG_WIDTH, IMG_HEIGHT, 3), classes=classes, alpha=alpha)

In [ ]:
# #os.environ["WANDB_MODE"] = "online"
# def train_model(model):

#         # solve issue from: https://github.com/wandb/wandb/issues/3536
#         # if len(wandb.patched["tensorboard"]) > 0:
#         #         wandb.tensorboard.unpatch()
                
#         # Configure Tensorboard root log directory to read the debugging information
#         #wandb.tensorboard.patch(root_logdir=root_logdir)
#         # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
#         # wandb.init(
#         #         # Set the project where this run will be logged
#         #         project=PROJECT, 
#         #         # Track hyperparameters and run metadata
#         #         #config={
#         #         #"learning_rate": LR,
#         #         #"epochs": EPOCHS,
#         #         #},
#         #         sync_tensorboard=True
#         #         )



#         # config = wandb.config
#         # # Specify the configuration variables
#         # config.batch_size = BATCH_SIZE
#         # config.dropout =DROPOUT
#         # config.learn_rate = LR
#         # #config.decay = 1e-6
#         # #config.momentum = 0.9
#         # config.epochs = EPOCHS
#         # config.classes = classes
        

#         # enable Tensorflow Debugging
#         #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
#         #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

#         #model = mobilenet
#         model.compile(optimizer='adam',
#                         loss='sparse_categorical_crossentropy',
#                         metrics=['accuracy'])

#         logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#         #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=10, update_freq="epoch") #, profile_batch="10, 20")
#         tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir) #, histogram_freq=1)
#         #wandb_callback = WandbCallback()# input_type="image", labels=labels) #, validation_data = val_ds.as_numpy_iterator())

#         early_stopping = EarlyStopping(monitor="val_accuracy", patience= early_stopping_patience)

#         #checkpoint = ModelCheckpoint("my_tiny_model", save_weights_only=True)

#         callbacks =[
#                 #tensorboard_callback,
#                 #wandb_callback,
#                 #WandbMetricsLogger(),
#                 #checkpoint,
#                 #early_stopping
#         ]

#         history = model.fit(train_ds,
#                 epochs=EPOCHS, 
#                 validation_data=val_ds, 
#                 callbacks=callbacks
#         )

#         # wandb.log({
#         #         "loss": history.history["loss"],
#         #         "accuracy": history.history["accuracy"],
#         #         "val_loss": history.history["val_loss"],
#         #         "val_accuracy": history.history["val_accuracy"],                                
#         # })
        
#         #wandb.finish()
#         return history, model
